In [1]:
import composeml as cp

df = cp.demos.load_transactions()

df[df.columns[:7]].head()

,transaction_id,session_id,product_id,amount,customer_id
transaction_time,,,,,
2014-01-01 03:13:51,190,14,5,120.52,1
2014-01-01 11:04:42,350,19,3,65.43,3
2014-01-02 11:44:35,254,11,5,128.51,4
2014-01-02 17:12:39,337,16,2,105.15,2
2014-01-02 17:46:20,177,29,5,65.11,1


In [2]:
def total_spent(df):
    total = df['amount'].sum()
    return total

In [3]:
label_maker = cp.LabelMaker(
    target_entity="customer_id",
    time_index="transaction_time",
    labeling_function=total_spent,
    window_size="1h",
)

In [4]:
labels = label_maker.search(
    df.sort_values('transaction_time'),
    num_examples_per_instance=-1,
    gap=1,
    verbose=True,
)

labels.head()

Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| customer_id: 125/125 


,customer_id,cutoff_time,my_labeling_function
label_id,,,
0,1,2014-01-01 04:13:51,65.11
1,1,2014-01-03 15:41:34,101.08
2,1,2014-01-05 11:46:10,16.78
3,1,2014-01-06 09:54:58,108.16
4,1,2014-01-08 08:54:02,48.33


In [5]:
labels = labels.threshold(300)

labels.head()

,customer_id,cutoff_time,my_labeling_function
label_id,,,
0,1,2014-01-01 04:13:51,False
1,1,2014-01-03 15:41:34,True
2,1,2014-01-05 11:46:10,False
3,1,2014-01-06 09:54:58,True
4,1,2014-01-08 08:54:02,False


In [6]:
labels = labels.apply_lead('1h')

labels.head()

,customer_id,cutoff_time,my_labeling_function
label_id,,,
0,1,2014-01-01 03:13:51,False
1,1,2014-01-03 14:41:34,True
2,1,2014-01-05 10:46:10,False
3,1,2014-01-06 08:54:58,True
4,1,2014-01-08 07:54:02,False


In [7]:
labels.describe()

Label Distribution
------------------
False      75
True       50
Total:    125


Settings
--------
num_examples_per_instance    25
minimum_data                 1h
window_size                  1h
gap                           1


Transforms
----------
1. threshold
  - value:    100

2. apply_lead
  - value:    1h

